# IEMAP TUTORIAL
**Python example code to use as a walkthrough in using iemap API** <br/> 
<img src="https://iemap-api.readthedocs.io/en/latest/_static/iemap_logo_nobg.png"></img>
### Full documentation is available [here](https://iemap-api.readthedocs.io/en/latest/index.html)
> Note. API and its documentation is still in development. For any issue please report at ... 

## Important notes before starting
- IEMAP API are implemented by a REST API architecture build-up by some endpoints that require authentication
- **Prior to using IEMAP API users need to register [here](https://ai4mat.enea.it/dashboard/)**
- It is possible to upload the project's metadata and files using the user interface (UI) at the link above or use the python module here described (**account registration is available only using the UI)**
- The UI at the link above is temporary, a more complete and functional interface is in 

>## Install IEMAP Python module using `pip install iemap` 
**Python version >= 3.10 is required**

In [2]:
# You can install within jupyter notebook using
# if this is a first installation
# ! pip install iemap
# or to upgrade to latest release
# ! pip install --upgrade  iemap

In [8]:
# import usefull modules
import json
import getpass
from IPython.display import display
import pandas as pd
from pprint import pprint
# from iemap_api import *
from iemap import IEMAP

In [5]:
# To show the installed version on iemap module
# !pip show iemap

### Step 1 - define account and login 
> **This is a mandatory step**, if you don't have an account yet register [here](https://ai4mat.enea.it/dashboard/).

In [6]:
# define user credentials
user="sergio.ferlito@enea.it"
print(f"\nType password for user: {user}")
# type in your password in a masked input field
pwd=getpass.getpass()
api = IEMAP(user,pwd)
# login 
api.login()


Type password for user: sergio.ferlito@enea.it


 ········


User successfully logged in!


### Step 2 - Start defining a new project metadata
> Project metadata have to be provided using a JSON file or python dict with a "valid" schema. Follow this [link](https://iemap-api.readthedocs.io/en/latest/getstart_partner.html#create-metadata) for the details.

> #### Note that   
> - Main fields are: projects*,material*, process*, parameters, properties   
> - "project" is build up by "name" and "label", both strings ("name" is required)   
> - "material" is an object with several sub-fields, the only required is "formula"
> - "material" **<sup>1</sup>** has subfields"input" and "output" both having a nested object made up by "cell" (List of List), "lattice" (Object/Dict), "sites" (List of List) and "species" (List) 
> - "process" is an object with fields "method" and "isExperiment" mandatory
> - "parameters" and "properties" are both a List of objects made up by two fields "name"(string)  and "value"(string or numeric)

> ***denotes mandatory fields**
> **<sup>1</sup>** more on this in the example_metadata below

### Step 3 - Familiarize with the required JSON schema

In [11]:
# load json example metadata
# Opening JSON file
f = open("metadata_example.json")
example_metadata = json.load(f)

In [12]:
pprint(example_metadata)

{'material': {'formula': 'Mn6Na12Ni6O24',
              'input': {'cell': [[9.2958, 0.0, 0.0],
                                 [-3.0986, 5.3669, 0.0],
                                 [0.0, 0.0, 10.8981]],
                        'lattice': {'a': 9.2958,
                                    'alpha': 90.0,
                                    'b': 6.197171739592183,
                                    'beta': 120.0001508506647,
                                    'c': 10.8981,
                                    'gamma': 90.0},
                        'sites': [[0.0, 1.789, 8.174],
                                  [1.549, 0.894, 2.725],
                                  [0.0, 0.0, 0.0],
                                  [0.0, 0.0, 5.449],
                                  [1.549, 0.894, -0.954],
                                  [0.0, 1.789, 4.495],
                                  [0.0, 1.789, 0.954],
                                  [1.549, 0.894, 6.403],
                           

In [13]:
# You can check if metadata has a valid schema with the code below
api.metadata=example_metadata
print("Metadata check passed" if  not api.check_metadata() else "Metadata check not passed")

Metadata check passed


In [14]:
# Check metadata again, this time removing a required field as project name
example_metadata['project']['name']=None
print("Metadata check passed" if  not api.check_metadata() else "Metadata check not passed")

The metadata provided does note have a valid schema.
Please check the error details below:
 1 validation error for newProject
project -> name
  none is not an allowed value (type=type_error.none.not_allowed)
Metadata check not passed


In [15]:
# reset metadata before going on as it will be provided using a JSON file
api.metadata=None

### Step 4 - Save a new project 
> Having defined project metadata from a JSON file or as Python dict, to save it on DB use `.save` method.   
> After succesfully saved a new project, the API return an ObjectID that uniquely define the saved project, this id can be retrieved later on using `.get_id()` method.   
> No id is defined if not explicetly set with `.set_id()` or if no new project has been already saved.

In [16]:
# save project metadata as defined in json file
api.save(metadata='./metadata_example.json')

Document correctly inserted with ObjectID = 63872c25051efb193a4c3ecb


### Step 5 - Show current project id 
> **Initially, after login, no id is set**, so you can either:
    1. use `.set_id` and provide a valid id (e.g. from table above)
    2. use `.save`, this will save a new project with its metadata and, evenutally, its files. This will set the project as the current one with the id returned from DB.  
    <br>
    <br> 
    
    **To add project files to a previosuly saved project use `save_project_files`**

In [17]:
# Show current project id
api.get_id()

Current project id is 63872c25051efb193a4c3ecb


### Step 6 - Show user projects
> Note, below are only example (random) data

In [18]:
# show user saved projects
api.my_projects()

[{'id': '637a0e4be19d0faf6cf7e6d6',
  'iemap_id': 'iemap-bd17d1',
  'project_name': 'Materials for Batteries',
  'date_creation': '2022-11-20T11:23:55.955000',
  'experiment': False,
  'material': 'Mn4Na12O24Ti8'},
 {'id': '637a0e4ce19d0faf6cf7e6d7',
  'iemap_id': 'iemap-f812e3',
  'project_name': 'Materials for Batteries',
  'date_creation': '2022-11-20T11:23:56.165000',
  'experiment': False,
  'material': 'Mn4Na12O24Ti8'},
 {'id': '637a0effe19d0faf6cf7e6d8',
  'iemap_id': 'iemap-0c167c',
  'project_name': 'Materials for Batteries',
  'date_creation': '2022-11-20T11:26:55.739000',
  'experiment': False,
  'material': 'Mn6Na12Ni6O24'},
 {'id': '637a0effe19d0faf6cf7e6d9',
  'iemap_id': 'iemap-b22ff8',
  'project_name': 'Materials for Batteries',
  'date_creation': '2022-11-20T11:26:55.952000',
  'experiment': False,
  'material': 'Mn6Na12Ni6O24'},
 {'id': '637a3bbbe19d0faf6cf7e6da',
  'iemap_id': 'iemap-13b771',
  'project_name': 'Materials for Batteries',
  'date_creation': '2022-11-2

#### List projects in tabular format (Pandas DataFrame)

In [19]:
df_proj=pd.DataFrame(api.my_projects())
df_proj

,id,iemap_id,project_name,date_creation,experiment,material
0,637a0e4be19d0faf6cf7e6d6,iemap-bd17d1,Materials for Batteries,2022-11-20T11:23:55.955000,False,Mn4Na12O24Ti8
1,637a0e4ce19d0faf6cf7e6d7,iemap-f812e3,Materials for Batteries,2022-11-20T11:23:56.165000,False,Mn4Na12O24Ti8
2,637a0effe19d0faf6cf7e6d8,iemap-0c167c,Materials for Batteries,2022-11-20T11:26:55.739000,False,Mn6Na12Ni6O24
3,637a0effe19d0faf6cf7e6d9,iemap-b22ff8,Materials for Batteries,2022-11-20T11:26:55.952000,False,Mn6Na12Ni6O24
4,637a3bbbe19d0faf6cf7e6da,iemap-13b771,Materials for Batteries,2022-11-20T14:37:47.295000,False,Mn6Na12Ni6O24
5,637a3bbbe19d0faf6cf7e6db,iemap-58f8f8,Materials for Batteries,2022-11-20T14:37:47.518000,False,Mn6Na12Ni6O24
6,6385ba9c051efb193a4c3ec1,iemap-cb042d,Materials for Batteries,2022-11-29T07:54:04.600000,False,Mn6Na12Ni6O24
7,6385bb3c051efb193a4c3ec2,iemap-489402,Materials for Batteries,2022-11-29T07:56:44.636000,False,Mn6Na12Ni6O24
8,6385bb6f051efb193a4c3ec3,iemap-b2b0df,Materials for Batteries,2022-11-29T07:57:35.880000,False,Mn6Na12Ni6O24
9,6385bbe3051efb193a4c3ec4,iemap-564ecf,Materials for Batteries,2022-11-29T07:59:31.267000,False,Mn6Na12Ni6O24


In [20]:
# grab a valid project id from previous printed list and set one
id_first_proj=df_proj.iloc[0,0]
api.set_id(id_first_proj)

Updated current project id to 637a0e4be19d0faf6cf7e6d6!!


In [21]:
# verify new id was correctly set
api.get_id()

Current project id is 637a0e4be19d0faf6cf7e6d6


### Step 7 - Add project file to existing project
> For each user project is possible to define one or more file to upload on Server.     
> Note: **If the file to upload is already present on server then it is not actually over written (on file system) but it is added as field on database.<br/>A note stating "(File already present on File System)" in this case is diplayed.**   
> **Only some file types are allowed as: PDF,CSV, XLS, XLSX, TXT, CIF or DOC.**

In [22]:
# add project file to a previously save project
_= api.save_project_files(project_files=["../migration/BANDs/10_bandsdown.dat"], show_debug_info=True)

Adding file 10_bandsdown.dat (1.21KB)...Done! (File already present on File System)
Saving project file took 0.1410 seconds


### Step 8 - Define a new project metadata (with its associated files in any)
> You can save metadata and files in a unique step using `.save` and providing both ***metadata*** and ***list_proj_files***

In [23]:
# define a list of valid files path
proj_files = ["../migration/CIFs/10.cif", "../migration/BANDs/1_bandsdown.dat"]

# save project and file
api.save(metadata="./metadata_example.json", list_proj_files=proj_files, show_debug_info=True)

Document correctly inserted with ObjectID = 63872c37051efb193a4c3ecc
Adding file 10.cif (5.69KB)...Done!
Adding file 1_bandsdown.dat (1.21KB)...Done!
Saving project file took 0.1580 seconds


### Step 9 - Query a specific Project id
> **All project and its id are available as showed at step 6**

In [24]:
id_last_proj=df_proj.iloc[-1,0]
data=api.query(id_last_proj)
data

[{'iemap_id': 'iemap-40d9f0',
  'provenance': {'email': '**********',
   'affiliation': 'ENEA',
   'createdAt': '2022-11-30T10:10:45.132000',
   'updatedAt': '2022-11-30T10:10:45.132000'},
  'project': {'name': 'Materials for Batteries', 'label': 'MB'},
  'process': {'isExperiment': False,
   'method': 'dft',
   'agent': {'name': 'Quantum Espresso', 'version': '6.7'}},
  'material': {'formula': 'Mn6Na12Ni6O24',
   'elements': ['Mn', 'Na', 'Ni', 'O'],
   'input': {'lattice': {'a': '9.2958',
     'b': '6.197171739592183',
     'c': '10.8981',
     'alpha': '90.0',
     'beta': '120.0001508506647',
     'gamma': '90.0'},
    'sites': [[0.0, 1.789, 8.174],
     [1.549, 0.894, 2.725],
     [0.0, 0.0, 0.0],
     [0.0, 0.0, 5.449],
     [1.549, 0.894, -0.954],
     [0.0, 1.789, 4.495],
     [0.0, 1.789, 0.954],
     [1.549, 0.894, 6.403],
     [6.197, 1.789, 8.174],
     [7.747, 0.894, 2.725],
     [6.197, 0.0, 0.0],
     [6.197, 0.0, 5.449],
     [6.197, 1.789, 0.954],
     [7.747, 0.894, 6.

### Step 10 - Show ONLY files for a specific project

In [25]:
# project files
if "files" in data[0]:
    files_df=pd.json_normalize(data, record_path='files')
    display(files_df)
else:
    print(f"No files associated to project {data[0]['iemap_id']}")

No files associated to project iemap-40d9f0


### Step 11  - Add project files to current project 
(or define another project id and add files to this new project id)

In [26]:
additional_files_proj = [
    "../migration/BANDs/1_bandsup.dat",
    "../migration/BANDs/7_bandsup.dat",
    "../migration/BANDs/1_bandsdown.dat",
    "../migration/QE/0.out",
    "../migration/QE/0.in",
    "../migration/CIFs/10.cif",
]

# save additional project files to current ID
_ =api.save_project_files(project_files=additional_files_proj)

Adding file 1_bandsup.dat (1.21KB)...Done!
Adding file 7_bandsup.dat (1.21KB)...Done!
Adding file 1_bandsdown.dat (1.21KB)...Done! (File already present on File System)
Adding file 0.out (6.54MB)...Done!
Adding file 0.in (4.15KB)...Done!
Adding file 10.cif (5.69KB)...Done! (File already present on File System)
Saving project file took 1.7720 seconds


### Step 12 - Associate a file to a specific property of material inside a project
> For each project is possible to associate one or more file related to the project as a whole,   
but is also possible to upload a file associtaed to a specific property of the material used in the project   
using `.save_property_files` and provinding as argument the property name and its related fiel as below

In [27]:
# First Query a specific project

result=api.query(id_first_proj)
if result != []:
    display(result[0]["properties"])

[{'name': 'volume', 'value': 546.81206681414},
 {'name': 'number_of_atoms', 'value': 48.0},
 {'name': 'number_of_electrons', 'value': 408.0},
 {'name': 'total_energy', 'value': -51149.096020383},
 {'name': 'formation_energy_per_atom', 'value': -2.936476648619949},
 {'name': 'energy_gap', 'value': -2.943662394163},
 {'name': 'fermi_energy', 'value': 10.4157},
 {'name': 'absolute_magnetization',
  'value': 23.57,
  'file': '23eb630d314515b1fd0326ac40fd128be7fbfa87.cif'},
 {'name': 'total_magnetization', 'value': 16.0}]

In [28]:
# set the project id and the property to which associate a file
api.set_id(id_first_proj)
api.save_property_files({"absolute_magnetization": "../migration/CIFs/0.cif"}, show_debug_info=True)

Updated current project id to 637a0e4be19d0faf6cf7e6d6!!
Adding file 0.cif (5.66KB)...Done! (File already present on File System)


In [29]:
# CHECK IF FILE HAS BEEN CORRECTLY ADDED
dataExistingProj=api.query(id_first_proj)

In [30]:
if "files" in dataExistingProj[0]:
    files_df=pd.json_normalize(dataExistingProj, record_path='files')
    display(files_df)
else:
    print(f"No files associated to project {data[0]['iemap_id']}")

,hash,name,extention,size,createdAt,updatedAt
0,54b56034096881366c52743ca922ad8194710830,10_bandsdown.dat,dat,1.213 KB,2022-11-30T08:30:47.312000,2022-11-30T08:30:47.312000


### Step 13 - Download a project (property) file
> Knowing the full file name = hash.ext is possible to download the file using `.download_file` (uploaded by the logged in user or not)

In [31]:
file_hash=files_df.iloc[0,0]
file_ext=str(files_df.iloc[0,1]).split(".")[-1]
full_file_hash=file_hash +"."+file_ext
print(full_file_hash)

54b56034096881366c52743ca922ad8194710830.dat


In [32]:
local_path_downloade_file='./downloaded_file.dat'
api.download_file(full_file_hash,local_path_downloade_file)

Download started, please wait....Done!


### Step 14 - Delete a file from project
> To delete a file from a project use `delete_project_file` method with argument the hash (only) of the file to be removed   
> Note: **You can delete from File System only files you uploaded and if not used in any other project**

In [33]:
api.delete_project_file("6b39ac4704eb5e36707766fc92e442b029d80ba8")

Deleting file 6b39ac4704eb5e36707766fc92e442b029d80ba8
File 6b39ac4704eb5e36707766fc92e442b029d80ba8.dat (1.214 KB), removed from project and from File System
